In [1]:
import os

# Please change the following to your own PAPER api key and secret
# You can get them from https://alpaca.markets/
# Alternatively, you can set the APCA_API_KEY_ID and APCA_API_SECRET_KEY environment variables
api_key = os.getenv("ALPACA_API_KEY")
secret_key = os.getenv("ALPACA_API_SECRET")

#### We use paper environment for this example ####
paper = True # Please do not modify this. This example is for paper trading only.
####

# Below are the variables for development this documents
# Please do not change these variables
trade_api_url = None
trade_api_wss = None
data_api_url = None
stream_data_wss = None

In [2]:
from datetime import datetime, timedelta
from zoneinfo import ZoneInfo

import alpaca
from alpaca.trading.client import TradingClient
from alpaca.data.timeframe import TimeFrame, TimeFrameUnit
from alpaca.data.historical.corporate_actions import CorporateActionsClient
from alpaca.data.historical.stock import StockHistoricalDataClient
from alpaca.trading.stream import TradingStream
from alpaca.data.live.stock import StockDataStream

from alpaca.data.requests import (
    CorporateActionsRequest,
    StockBarsRequest,
    StockQuotesRequest,
    StockTradesRequest,
)
from alpaca.trading.requests import (
    ClosePositionRequest,
    GetAssetsRequest,
    GetOrdersRequest,
    LimitOrderRequest,
    MarketOrderRequest,
    StopLimitOrderRequest,
    StopLossRequest,
    StopOrderRequest,
    TakeProfitRequest,
    TrailingStopOrderRequest,
)
from alpaca.trading.enums import (
    AssetExchange,
    AssetStatus,
    OrderClass,
    OrderSide,
    OrderType,
    QueryOrderStatus,
    TimeInForce,
)

In [3]:
# to run async code in jupyter notebook
import nest_asyncio
nest_asyncio.apply()

In [4]:
# check version of alpaca-py
alpaca.__version__

'0.34.0'

In [5]:
# setup clients
trade_client = TradingClient(api_key=api_key, secret_key=secret_key, paper=paper, url_override=trade_api_url)

In [6]:
# check trading account
# You can check definition of each field in the following documents
# ref. https://docs.alpaca.markets/docs/account-plans
# ref. https://docs.alpaca.markets/reference/getaccount-1
acct = trade_client.get_account()
acct

{   'account_blocked': False,
    'account_number': 'PA3B3ROUWP8U',
    'accrued_fees': '0',
    'buying_power': '99378.64',
    'cash': '29.9',
    'created_at': datetime.datetime(2024, 12, 11, 11, 27, 22, 595572, tzinfo=TzInfo(UTC)),
    'crypto_status': <AccountStatus.ACTIVE: 'ACTIVE'>,
    'currency': 'USD',
    'daytrade_count': 1,
    'daytrading_buying_power': '0',
    'equity': '99348.74',
    'id': UUID('d3c62a03-a491-4d6a-b725-7f969e4d10c5'),
    'initial_margin': '49659.42',
    'last_equity': '98502.68',
    'last_maintenance_margin': '29541.83',
    'long_market_value': '99318.84',
    'maintenance_margin': '29795.65',
    'multiplier': '2',
    'non_marginable_buying_power': '47689.32',
    'options_approved_level': 2,
    'options_buying_power': '69553.08',
    'options_trading_level': 2,
    'pattern_day_trader': False,
    'pending_transfer_in': None,
    'pending_transfer_out': None,
    'portfolio_value': '99348.74',
    'regt_buying_power': '99378.64',
    'short_ma

In [7]:
# check account configuration
# ref. https://docs.alpaca.markets/reference/getaccountconfig-1
acct_config = trade_client.get_account_configurations()
acct_config

{   'dtbp_check': <DTBPCheck.ENTRY: 'entry'>,
    'fractional_trading': True,
    'max_margin_multiplier': '4',
    'max_options_trading_level': None,
    'no_shorting': False,
    'pdt_check': <PDTCheck.ENTRY: 'entry'>,
    'ptp_no_exception_entry': False,
    'suspend_trade': False,
    'trade_confirm_email': <TradeConfirmationEmail.ALL: 'all'>}

In [8]:
# set account configuration
# ref. https://docs.alpaca.markets/reference/patchaccountconfig-1
req = acct_config
req.fractional_trading = not req.fractional_trading # toggle fractional trading
acct_config_new = trade_client.set_account_configurations(req)
display(acct_config_new)

# revert changes
req = acct_config_new
req.fractional_trading = not req.fractional_trading # toggle fractional trading
acct_config_reverted = trade_client.set_account_configurations(req)
display(acct_config_reverted)

{   'dtbp_check': <DTBPCheck.ENTRY: 'entry'>,
    'fractional_trading': False,
    'max_margin_multiplier': '4',
    'max_options_trading_level': None,
    'no_shorting': False,
    'pdt_check': <PDTCheck.ENTRY: 'entry'>,
    'ptp_no_exception_entry': False,
    'suspend_trade': False,
    'trade_confirm_email': <TradeConfirmationEmail.ALL: 'all'>}

{   'dtbp_check': <DTBPCheck.ENTRY: 'entry'>,
    'fractional_trading': True,
    'max_margin_multiplier': '4',
    'max_options_trading_level': None,
    'no_shorting': False,
    'pdt_check': <PDTCheck.ENTRY: 'entry'>,
    'ptp_no_exception_entry': False,
    'suspend_trade': False,
    'trade_confirm_email': <TradeConfirmationEmail.ALL: 'all'>}

In [9]:
# get list of assets which are us_equity (default), active, and in NASDAQ
# ref. https://docs.alpaca.markets/reference/get-v2-assets-1
req = GetAssetsRequest(
  # asset_class=AssetClass.US_EQUITY,  # default asset_class is us_equity
  status=AssetStatus.ACTIVE,
  exchange=AssetExchange.NASDAQ,
)
assets = trade_client.get_all_assets(req)
assets[:2]

[{   'asset_class': <AssetClass.US_EQUITY: 'us_equity'>,
     'attributes': [],
     'easy_to_borrow': False,
     'exchange': <AssetExchange.NASDAQ: 'NASDAQ'>,
     'fractionable': False,
     'id': UUID('70836657-1936-415a-86c3-648aad919b47'),
     'maintenance_margin_requirement': 100.0,
     'marginable': False,
     'min_order_size': None,
     'min_trade_increment': None,
     'name': 'Able View Global Inc. Class B Ordinary Shares',
     'price_increment': None,
     'shortable': False,
     'status': <AssetStatus.ACTIVE: 'active'>,
     'symbol': 'ABLV',
     'tradable': False},
 {   'asset_class': <AssetClass.US_EQUITY: 'us_equity'>,
     'attributes': [],
     'easy_to_borrow': False,
     'exchange': <AssetExchange.NASDAQ: 'NASDAQ'>,
     'fractionable': False,
     'id': UUID('b9447a20-a641-4208-a690-7d6a0b49d271'),
     'maintenance_margin_requirement': 100.0,
     'marginable': False,
     'min_order_size': None,
     'min_trade_increment': None,
     'name': 'Magic Empire

In [10]:
# we will place orders which Alapca trading platform supports
# - order classes: simple, bracket, oco, oto
# - order types: market, limit, stop, stop_limit, trailing_stop
#
# please refer to the following documents for more details
# ref. https://docs.alpaca.markets/docs/orders-at-alpaca
# ref. https://docs.alpaca.markets/reference/postorder
#
# we will also use fractional trading capability of Alpaca trading platform in this example
# ref. https://docs.alpaca.markets/docs/fractional-trading

# we will place orders for symbol: SPY in this example
symbol = "SPY"

In [11]:
# simple, market order, fractional qty
# Alpaca trading platform support fractional trading by default
# you can specify:
# fractional qty (e.g. 0.01 qty) in the order request (which is shown in this example)
# or notional value (e.g. 100 USD) (which is in the next example)
#
# If you have an error of `qty must be integer`,
# please try to `Reset Account` of your paper account via the Alpaca Trading API dashboard
req = MarketOrderRequest(
    symbol = symbol,
    qty = 5.5,
    side = OrderSide.BUY,
    type = OrderType.MARKET,
    time_in_force = TimeInForce.DAY,
)
res = trade_client.submit_order(req)
res

{   'asset_class': <AssetClass.US_EQUITY: 'us_equity'>,
    'asset_id': UUID('b28f4066-5c6d-479b-a2af-85dc1a8f16fb'),
    'canceled_at': None,
    'client_order_id': '02bfc84c-a4de-4088-982e-ce88df2380f2',
    'created_at': datetime.datetime(2024, 12, 13, 9, 39, 30, 143494, tzinfo=TzInfo(UTC)),
    'expired_at': None,
    'extended_hours': False,
    'failed_at': None,
    'filled_at': None,
    'filled_avg_price': None,
    'filled_qty': '0',
    'hwm': None,
    'id': UUID('ecb91eaa-c418-405a-9028-91c07056163d'),
    'legs': None,
    'limit_price': None,
    'notional': None,
    'order_class': <OrderClass.SIMPLE: 'simple'>,
    'order_type': <OrderType.MARKET: 'market'>,
    'qty': '5.5',
    'replaced_at': None,
    'replaced_by': None,
    'replaces': None,
    'side': <OrderSide.BUY: 'buy'>,
    'status': <OrderStatus.PENDING_NEW: 'pending_new'>,
    'stop_price': None,
    'submitted_at': datetime.datetime(2024, 12, 13, 9, 39, 30, 143494, tzinfo=TzInfo(UTC)),
    'symbol': 'SPY

In [12]:
# simple, market order, notional
# Alpaca trading platform support fractional trading by default
# you can specify:
# fractional qty (e.g. 0.01 qty) in the order request (which is in the example above)
# or notional value (e.g. 100 USD) (which is in this example)
req = MarketOrderRequest(
    symbol = symbol,
    notional = 1.11,  # notional is specified in USD, here we specify $1.11
    side = OrderSide.BUY,
    type = OrderType.MARKET,
    time_in_force = TimeInForce.DAY,
)
res = trade_client.submit_order(req)
res

{   'asset_class': <AssetClass.US_EQUITY: 'us_equity'>,
    'asset_id': UUID('b28f4066-5c6d-479b-a2af-85dc1a8f16fb'),
    'canceled_at': None,
    'client_order_id': '848f0823-355d-4717-861e-a18847ce5024',
    'created_at': datetime.datetime(2024, 12, 13, 9, 39, 30, 453585, tzinfo=TzInfo(UTC)),
    'expired_at': None,
    'extended_hours': False,
    'failed_at': None,
    'filled_at': None,
    'filled_avg_price': None,
    'filled_qty': '0',
    'hwm': None,
    'id': UUID('41840213-cf32-4897-a5fe-8505e988ad04'),
    'legs': None,
    'limit_price': None,
    'notional': '1.11',
    'order_class': <OrderClass.SIMPLE: 'simple'>,
    'order_type': <OrderType.MARKET: 'market'>,
    'qty': None,
    'replaced_at': None,
    'replaced_by': None,
    'replaces': None,
    'side': <OrderSide.BUY: 'buy'>,
    'status': <OrderStatus.ACCEPTED: 'accepted'>,
    'stop_price': None,
    'submitted_at': datetime.datetime(2024, 12, 13, 9, 39, 30, 453585, tzinfo=TzInfo(UTC)),
    'symbol': 'SPY',
  

In [13]:
# simple, limit order, fractional qty
req = LimitOrderRequest(
    symbol = symbol,
    qty = 0.01,
    limit_price = 550.25,
    side = OrderSide.BUY,
    type = OrderType.LIMIT,
    time_in_force = TimeInForce.DAY,
)
res = trade_client.submit_order(req)
res

{   'asset_class': <AssetClass.US_EQUITY: 'us_equity'>,
    'asset_id': UUID('b28f4066-5c6d-479b-a2af-85dc1a8f16fb'),
    'canceled_at': None,
    'client_order_id': 'e7ecfc94-391e-4aa4-a759-0b87420b412c',
    'created_at': datetime.datetime(2024, 12, 13, 9, 39, 30, 762845, tzinfo=TzInfo(UTC)),
    'expired_at': None,
    'extended_hours': False,
    'failed_at': None,
    'filled_at': None,
    'filled_avg_price': None,
    'filled_qty': '0',
    'hwm': None,
    'id': UUID('2ab7b0c9-4d0f-41db-a597-7435a0118e19'),
    'legs': None,
    'limit_price': '550.25',
    'notional': None,
    'order_class': <OrderClass.SIMPLE: 'simple'>,
    'order_type': <OrderType.LIMIT: 'limit'>,
    'qty': '0.01',
    'replaced_at': None,
    'replaced_by': None,
    'replaces': None,
    'side': <OrderSide.BUY: 'buy'>,
    'status': <OrderStatus.ACCEPTED: 'accepted'>,
    'stop_price': None,
    'submitted_at': datetime.datetime(2024, 12, 13, 9, 39, 30, 762845, tzinfo=TzInfo(UTC)),
    'symbol': 'SPY',


In [14]:
# stop order
req = StopOrderRequest(
                    symbol = symbol,
                    qty = 1,
                    side = OrderSide.BUY,
                    time_in_force = TimeInForce.GTC,
                    stop_price = 600
                    )

res = trade_client.submit_order(req)
res

{   'asset_class': <AssetClass.US_EQUITY: 'us_equity'>,
    'asset_id': UUID('b28f4066-5c6d-479b-a2af-85dc1a8f16fb'),
    'canceled_at': None,
    'client_order_id': '3920573f-8ac5-4454-aa52-40979fcbefa7',
    'created_at': datetime.datetime(2024, 12, 13, 9, 39, 31, 69223, tzinfo=TzInfo(UTC)),
    'expired_at': None,
    'extended_hours': False,
    'failed_at': None,
    'filled_at': None,
    'filled_avg_price': None,
    'filled_qty': '0',
    'hwm': None,
    'id': UUID('8f6bfb8a-7fd3-435d-9c30-af02f6166b23'),
    'legs': None,
    'limit_price': None,
    'notional': None,
    'order_class': <OrderClass.SIMPLE: 'simple'>,
    'order_type': <OrderType.STOP: 'stop'>,
    'qty': '1',
    'replaced_at': None,
    'replaced_by': None,
    'replaces': None,
    'side': <OrderSide.BUY: 'buy'>,
    'status': <OrderStatus.PENDING_NEW: 'pending_new'>,
    'stop_price': '600',
    'submitted_at': datetime.datetime(2024, 12, 13, 9, 39, 31, 69223, tzinfo=TzInfo(UTC)),
    'symbol': 'SPY',
    

In [15]:
# stop limit order
req = StopLimitOrderRequest(
                    symbol = symbol,
                    qty = 1,
                    side = OrderSide.BUY,
                    time_in_force = TimeInForce.GTC,
                    limit_price = 550,
                    stop_price = 600
                    )

res = trade_client.submit_order(req)
res

{   'asset_class': <AssetClass.US_EQUITY: 'us_equity'>,
    'asset_id': UUID('b28f4066-5c6d-479b-a2af-85dc1a8f16fb'),
    'canceled_at': None,
    'client_order_id': 'dab47943-bf3c-4cd3-96c2-765a521a04e0',
    'created_at': datetime.datetime(2024, 12, 13, 9, 39, 31, 376216, tzinfo=TzInfo(UTC)),
    'expired_at': None,
    'extended_hours': False,
    'failed_at': None,
    'filled_at': None,
    'filled_avg_price': None,
    'filled_qty': '0',
    'hwm': None,
    'id': UUID('a45a64ed-1767-4de7-bfe5-5c3e9a918ada'),
    'legs': None,
    'limit_price': '550',
    'notional': None,
    'order_class': <OrderClass.SIMPLE: 'simple'>,
    'order_type': <OrderType.STOP_LIMIT: 'stop_limit'>,
    'qty': '1',
    'replaced_at': None,
    'replaced_by': None,
    'replaces': None,
    'side': <OrderSide.BUY: 'buy'>,
    'status': <OrderStatus.PENDING_NEW: 'pending_new'>,
    'stop_price': '600',
    'submitted_at': datetime.datetime(2024, 12, 13, 9, 39, 31, 376216, tzinfo=TzInfo(UTC)),
    'symbo

In [16]:
# bracket order with both stop loss and take profit
req = MarketOrderRequest(
                    symbol = symbol,
                    qty = 5,
                    side = OrderSide.BUY,
                    time_in_force = TimeInForce.DAY,
                    order_class = OrderClass.BRACKET,
                    take_profit = TakeProfitRequest(limit_price=700),
                    stop_loss = StopLossRequest(stop_price=300)
)
res = trade_client.submit_order(req)
res

{   'asset_class': <AssetClass.US_EQUITY: 'us_equity'>,
    'asset_id': UUID('b28f4066-5c6d-479b-a2af-85dc1a8f16fb'),
    'canceled_at': None,
    'client_order_id': '360c43b3-525f-484d-b79f-7d7165141df0',
    'created_at': datetime.datetime(2024, 12, 13, 9, 39, 31, 682767, tzinfo=TzInfo(UTC)),
    'expired_at': None,
    'extended_hours': False,
    'failed_at': None,
    'filled_at': None,
    'filled_avg_price': None,
    'filled_qty': '0',
    'hwm': None,
    'id': UUID('d4c5bbb4-163e-43d5-a9e8-e1e9703568a8'),
    'legs': [   {   'asset_class': <AssetClass.US_EQUITY: 'us_equity'>,
                    'asset_id': UUID('b28f4066-5c6d-479b-a2af-85dc1a8f16fb'),
                    'canceled_at': None,
                    'client_order_id': 'e6dfc577-caa5-443b-a3ba-516999b0b6e6',
                    'created_at': datetime.datetime(2024, 12, 13, 9, 39, 31, 682767, tzinfo=TzInfo(UTC)),
                    'expired_at': None,
                    'extended_hours': False,
                  

In [17]:
# oto order with stop loss
req = LimitOrderRequest(
                    symbol = symbol,
                    qty = 1,
                    limit_price = 500,
                    side = OrderSide.BUY,
                    time_in_force = TimeInForce.DAY,
                    Class = OrderClass.OTO,
                    stop_loss = StopLossRequest(stop_price = 300)
                    )

res = trade_client.submit_order(req)
res

{   'asset_class': <AssetClass.US_EQUITY: 'us_equity'>,
    'asset_id': UUID('b28f4066-5c6d-479b-a2af-85dc1a8f16fb'),
    'canceled_at': None,
    'client_order_id': 'ecf0dbb5-6310-41ed-bc4e-fd1e493f2f55',
    'created_at': datetime.datetime(2024, 12, 13, 9, 39, 32, 25417, tzinfo=TzInfo(UTC)),
    'expired_at': None,
    'extended_hours': False,
    'failed_at': None,
    'filled_at': None,
    'filled_avg_price': None,
    'filled_qty': '0',
    'hwm': None,
    'id': UUID('7d581a5e-8cd6-4f23-9421-c2bd1ebe54e9'),
    'legs': None,
    'limit_price': '500',
    'notional': None,
    'order_class': <OrderClass.SIMPLE: 'simple'>,
    'order_type': <OrderType.LIMIT: 'limit'>,
    'qty': '1',
    'replaced_at': None,
    'replaced_by': None,
    'replaces': None,
    'side': <OrderSide.BUY: 'buy'>,
    'status': <OrderStatus.PENDING_NEW: 'pending_new'>,
    'stop_price': None,
    'submitted_at': datetime.datetime(2024, 12, 13, 9, 39, 32, 25417, tzinfo=TzInfo(UTC)),
    'symbol': 'SPY',
  

In [18]:
# oco limit order
req = LimitOrderRequest(
                    symbol = symbol,
                    qty = 1,
                    limit_price = 500,
                    side = OrderSide.BUY,
                    time_in_force = TimeInForce.DAY,
                    Class = OrderClass.OCO
                    )

res = trade_client.submit_order(req)
res

{   'asset_class': <AssetClass.US_EQUITY: 'us_equity'>,
    'asset_id': UUID('b28f4066-5c6d-479b-a2af-85dc1a8f16fb'),
    'canceled_at': None,
    'client_order_id': 'aedb8668-3731-4ed1-b895-42c88b627a11',
    'created_at': datetime.datetime(2024, 12, 13, 9, 39, 32, 432380, tzinfo=TzInfo(UTC)),
    'expired_at': None,
    'extended_hours': False,
    'failed_at': None,
    'filled_at': None,
    'filled_avg_price': None,
    'filled_qty': '0',
    'hwm': None,
    'id': UUID('d84833ca-609b-447b-bc0f-e480cdfddf48'),
    'legs': None,
    'limit_price': '500',
    'notional': None,
    'order_class': <OrderClass.SIMPLE: 'simple'>,
    'order_type': <OrderType.LIMIT: 'limit'>,
    'qty': '1',
    'replaced_at': None,
    'replaced_by': None,
    'replaces': None,
    'side': <OrderSide.BUY: 'buy'>,
    'status': <OrderStatus.PENDING_NEW: 'pending_new'>,
    'stop_price': None,
    'submitted_at': datetime.datetime(2024, 12, 13, 9, 39, 32, 432380, tzinfo=TzInfo(UTC)),
    'symbol': 'SPY',


In [21]:
# trailing stop order
req = TrailingStopOrderRequest(
                    symbol = symbol,
                    qty = 1,
                    side = OrderSide.SELL,
                    time_in_force = TimeInForce.GTC,
                    trail_percent = 0.20 # you can also use trail_price instead of trail_percent
                    )

res = trade_client.submit_order(req)
res

{   'asset_class': <AssetClass.US_EQUITY: 'us_equity'>,
    'asset_id': UUID('b28f4066-5c6d-479b-a2af-85dc1a8f16fb'),
    'canceled_at': None,
    'client_order_id': 'bed7b27c-dd89-46d2-9928-c5f83492fecc',
    'created_at': datetime.datetime(2024, 12, 13, 9, 40, 36, 999421, tzinfo=TzInfo(UTC)),
    'expired_at': None,
    'extended_hours': False,
    'failed_at': None,
    'filled_at': None,
    'filled_avg_price': None,
    'filled_qty': '0',
    'hwm': '606.24',
    'id': UUID('418d8bb9-b935-498d-a49d-eeebc0813d9f'),
    'legs': None,
    'limit_price': None,
    'notional': None,
    'order_class': <OrderClass.SIMPLE: 'simple'>,
    'order_type': <OrderType.TRAILING_STOP: 'trailing_stop'>,
    'qty': '1',
    'replaced_at': None,
    'replaced_by': None,
    'replaces': None,
    'side': <OrderSide.SELL: 'sell'>,
    'status': <OrderStatus.NEW: 'new'>,
    'stop_price': '605.02752',
    'submitted_at': datetime.datetime(2024, 12, 13, 9, 40, 36, 999421, tzinfo=TzInfo(UTC)),
    'symb

In [22]:
# get a list of orders including closed (e.g. filled) orders by specifying symbol
req = GetOrdersRequest(
    status = QueryOrderStatus.ALL,
    symbols = [symbol]
)
orders = trade_client.get_orders(req)
orders

[{   'asset_class': <AssetClass.US_EQUITY: 'us_equity'>,
     'asset_id': UUID('b28f4066-5c6d-479b-a2af-85dc1a8f16fb'),
     'canceled_at': None,
     'client_order_id': 'bed7b27c-dd89-46d2-9928-c5f83492fecc',
     'created_at': datetime.datetime(2024, 12, 13, 9, 40, 36, 999421, tzinfo=TzInfo(UTC)),
     'expired_at': None,
     'extended_hours': False,
     'failed_at': None,
     'filled_at': None,
     'filled_avg_price': None,
     'filled_qty': '0',
     'hwm': '606.24',
     'id': UUID('418d8bb9-b935-498d-a49d-eeebc0813d9f'),
     'legs': None,
     'limit_price': None,
     'notional': None,
     'order_class': <OrderClass.SIMPLE: 'simple'>,
     'order_type': <OrderType.TRAILING_STOP: 'trailing_stop'>,
     'qty': '1',
     'replaced_at': None,
     'replaced_by': None,
     'replaces': None,
     'side': <OrderSide.SELL: 'sell'>,
     'status': <OrderStatus.NEW: 'new'>,
     'stop_price': '605.02752',
     'submitted_at': datetime.datetime(2024, 12, 13, 9, 40, 36, 999421, tzin

In [23]:
# see all open orders
req = GetOrdersRequest(
    status = QueryOrderStatus.OPEN,
    symbols = [symbol]
)
open_orders = trade_client.get_orders(req)
open_orders

[{   'asset_class': <AssetClass.US_EQUITY: 'us_equity'>,
     'asset_id': UUID('b28f4066-5c6d-479b-a2af-85dc1a8f16fb'),
     'canceled_at': None,
     'client_order_id': 'bed7b27c-dd89-46d2-9928-c5f83492fecc',
     'created_at': datetime.datetime(2024, 12, 13, 9, 40, 36, 999421, tzinfo=TzInfo(UTC)),
     'expired_at': None,
     'extended_hours': False,
     'failed_at': None,
     'filled_at': None,
     'filled_avg_price': None,
     'filled_qty': '0',
     'hwm': '606.24',
     'id': UUID('418d8bb9-b935-498d-a49d-eeebc0813d9f'),
     'legs': None,
     'limit_price': None,
     'notional': None,
     'order_class': <OrderClass.SIMPLE: 'simple'>,
     'order_type': <OrderType.TRAILING_STOP: 'trailing_stop'>,
     'qty': '1',
     'replaced_at': None,
     'replaced_by': None,
     'replaces': None,
     'side': <OrderSide.SELL: 'sell'>,
     'status': <OrderStatus.NEW: 'new'>,
     'stop_price': '605.02752',
     'submitted_at': datetime.datetime(2024, 12, 13, 9, 40, 36, 999421, tzin

In [24]:
# cancel all open orders
trade_client.cancel_orders()

[{   'body': None,
     'id': UUID('418d8bb9-b935-498d-a49d-eeebc0813d9f'),
     'status': 200}]

In [25]:
# get all open positions
# ref. https://docs.alpaca.markets/reference/getallopenpositions-1
positions = trade_client.get_all_positions()
positions

[{   'asset_class': <AssetClass.US_EQUITY: 'us_equity'>,
     'asset_id': UUID('4ce9353c-66d1-46c2-898f-fce867ab0247'),
     'asset_marginable': True,
     'avg_entry_price': '139.711926',
     'avg_entry_swap_rate': None,
     'change_today': '0.0097568079219455',
     'cost_basis': '100173.450942',
     'current_price': '138.68',
     'exchange': <AssetExchange.NASDAQ: 'NASDAQ'>,
     'lastday_price': '137.34',
     'market_value': '99433.56',
     'qty': '717',
     'qty_available': '717',
     'side': <PositionSide.LONG: 'long'>,
     'swap_rate': None,
     'symbol': 'NVDA',
     'unrealized_intraday_pl': '960.78',
     'unrealized_intraday_plpc': '0.0097568079219455',
     'unrealized_pl': '-739.890942',
     'unrealized_plpc': '-0.007386098163159',
     'usd': None}]

In [26]:
# get positions by symbol
# ref. https://docs.alpaca.markets/reference/getopenposition-1
position = trade_client.get_open_position(symbol_or_asset_id=symbol)
position


APIError: {"code":40410000,"message":"position does not exist"}

In [34]:
# get positions by asset_id
trade_client.get_open_position(symbol_or_asset_id=position.asset_id)

NameError: name 'position' is not defined

In [27]:
# close the position with specifying qty
# ref. https://docs.alpaca.markets/reference/deleteopenposition-1
trade_client.close_position(
    symbol_or_asset_id = symbol,
    close_options = ClosePositionRequest(
        qty = "0.01",
    )
)

APIError: {"code":40410000,"message":"position not found: SPY"}

In [28]:
# subscribe trade updates
trade_stream_client = TradingStream(api_key, secret_key, paper=paper, url_override = trade_api_wss)

async def trade_updates_handler(data):
    print(data)

trade_stream_client.subscribe_trade_updates(trade_updates_handler)
trade_stream_client.run()

trading stream websocket error, restarting  connection: sent 1011 (unexpected error) keepalive ping timeout; no close frame received
error during websocket communication: [Errno 11001] getaddrinfo failed
Traceback (most recent call last):
  File "c:\Users\chavv\anaconda\envs\stock-market\Lib\site-packages\alpaca\trading\stream.py", line 171, in _run_forever
    await self._start_ws()
  File "c:\Users\chavv\anaconda\envs\stock-market\Lib\site-packages\alpaca\trading\stream.py", line 131, in _start_ws
    await self._connect()
  File "c:\Users\chavv\anaconda\envs\stock-market\Lib\site-packages\alpaca\trading\stream.py", line 60, in _connect
    self._ws = await websockets_legacy.connect(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\chavv\anaconda\envs\stock-market\Lib\site-packages\websockets\legacy\client.py", line 659, in __await_impl_timeout__
    return await asyncio.wait_for(self.__await_impl__(), self.open_timeout)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

keyboard interrupt, bye


In [ ]:
# setup stock historical data client
stock_historical_data_client = StockHistoricalDataClient(api_key, secret_key, url_override = data_api_url)

In [ ]:
# get historical bars by symbol
# ref. https://docs.alpaca.markets/reference/stockbars-1
now = datetime.now(ZoneInfo("America/New_York"))
req = StockBarsRequest(
    symbol_or_symbols = [symbol],
    timeframe=TimeFrame(amount = 1, unit = TimeFrameUnit.Hour), # specify timeframe
    start = now - timedelta(days = 5),                          # specify start datetime, default=the beginning of the current day.
    # end_date=None,                                        # specify end datetime, default=now
    limit = 2,                                               # specify limit
)
stock_historical_data_client.get_stock_bars(req).df

In [ ]:
# get historical trades by symbol
req = StockTradesRequest(
    symbol_or_symbols = [symbol],
    start = now - timedelta(days = 5),                          # specify start datetime, default=the beginning of the current day.
    # end=None,                                             # specify end datetime, default=now
    limit = 2,                                                # specify limit
)
stock_historical_data_client.get_stock_trades(req).df

In [ ]:
# get historical quotes by symbol
req = StockQuotesRequest(
    symbol_or_symbols = [symbol],
    start = now - timedelta(days = 5),                      # specify start datetime, default=the beginning of the current day.
    # end=None,                                             # specify end datetime, default=now
    limit = 2,                                              # specify limit
)
stock_historical_data_client.get_stock_quotes(req).df

In [ ]:
# get latest quotes by symbol
req = StockQuotesRequest(
    symbol_or_symbols = [symbol],
)
res = stock_historical_data_client.get_stock_latest_quote(req)
res

In [ ]:
stock_data_stream_client = StockDataStream(api_key, secret_key, url_override = stream_data_wss)

async def stock_data_stream_handler(data):
    print(data)

symbols = [symbol]

stock_data_stream_client.subscribe_quotes(stock_data_stream_handler, *symbols)
stock_data_stream_client.subscribe_trades(stock_data_stream_handler, *symbols)

stock_data_stream_client.run()

In [ ]:
corporate_actions_client = CorporateActionsClient(api_key, secret_key)
corporate_actions_client.get_corporate_actions(CorporateActionsRequest(
    start=datetime(2020, 1, 1),
    symbols=[symbol]
)).df